In [1]:
data_dir = "bs_10k_10q_all_joined"

In [2]:
# Basic import to support interactive widgets in notebooks
import ipywidgets as widgets
from IPython.display import display, Markdown
from ipywidgets import interact, interact_manual

import pandas as pd
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

In [3]:
# imports from the secfsdstools package
from secfsdstools.c_index.searching import IndexSearch
from secfsdstools.c_index.companyindexreading import CompanyIndexReader
from secfsdstools.e_collector.reportcollecting import SingleReportCollector
from secfsdstools.d_container.databagmodel import JoinedDataBag
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, StmtRawFilter
from secfsdstools.e_presenter.presenting import StandardStatementPresenter
from secfsdstools.c_index.indexing import BaseReportIndexer

In [4]:
# initialize the search class
search = IndexSearch.get_index_search()

# create a list with all known forms
forms_list = sorted(search.dbaccessor.read_all_indexreports_df().form.unique().tolist())
stmt_list = ['BS', 'CF', 'CI', 'CP', 'EQ', 'IS', 'SI', 'UN']

2023-11-24 09:33:53,888 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2023-11-24 09:33:53,926 [INFO] updateprocess  Check if new report zip files are available...
2023-11-24 09:33:53,970 [INFO] updateprocess  check if there are new files to download from sec.gov ...
2023-11-24 09:33:54,779 [INFO] updateprocess  start to transform to parquet format ...
2023-11-24 09:33:54,779 [INFO] updateprocess  start to index parquet files ...


No rapid-api-key is set: 
If you are interested in daily updates, please have a look at https://rapidapi.com/hansjoerg.wingeier/api/daily-sec-financial-statement-dataset


In [15]:
from secfsdstools.e_filter.joinedfiltering import USDonlyFilter as USDonlyJoined
bs_bag = JoinedDataBag.load(data_dir)[USDonlyJoined()]

In [24]:
filtered = bs_bag.pre_num_df[bs_bag.pre_num_df.tag.isin(['AdditionalPaidInCapital','TreasuryStockValue'])]
counted = filtered[['adsh', 'coreg', 'report', 'ddate', 'uom', 'tag']].groupby(['adsh', 'coreg', 'report', 'ddate', 'uom']).count()
counted[counted.tag > 1]

,,,,,tag
adsh,coreg,report,ddate,uom,
0000003499-11-000024,,2,20110630,USD,2
0000003499-12-000005,,2,20111231,USD,2
0000003499-12-000011,,2,20120331,USD,2
0000003499-12-000020,,2,20120930,USD,2
0000003499-13-000006,,2,20121231,USD,2
...,...,...,...,...,...
0001883685-22-000026,,2,20220630,USD,2
0001883685-22-000036,,2,20220930,USD,2
0001883685-23-000013,,3,20221231,USD,2


In [23]:
bs_bag.pre_num_df[bs_bag.pre_num_df.tag.isin(["CommonStock","PreferredStock","AdditionalPaidInCapital","TreasuryStockValue","ContributedCapital"])].tag.value_counts()

AdditionalPaidInCapital    155555
TreasuryStockValue          87692
ContributedCapital             43
PreferredStock                 38
CommonStock                    26
Name: tag, dtype: int64

In [18]:
bs_bag.pre_num_df[bs_bag.pre_num_df.tag.isin(['RetainedEarningsAppropriated', 'RetainedEarningsUnappropriated','RetainedEarnings', 'RetainedEarningsAccumulatedDeficit'])].tag.value_counts()

RetainedEarningsAccumulatedDeficit    277399
RetainedEarningsUnappropriated          2689
RetainedEarningsAppropriated            2158
RetainedEarnings                          68
Name: tag, dtype: int64

In [21]:
# list all tags containing a certain string
[x for x in bs_bag.pre_num_df.tag.unique().tolist() if "PaidIn" in str(x)]

['PreferredStockIncludingAdditionalPaidInCapitalNetOfDiscount',
 'CommonStocksIncludingAdditionalPaidInCapital',
 'CommonStocksIncludingAdditionalPaidInCapitalNetOfDiscount',
 'AdditionalPaidInCapital',
 'AdditionalPaidInCapitalCommonStock',
 'AdditionalPaidInCapitalPreferredStock',
 'TemporaryEquityValueExcludingAdditionalPaidInCapital',
 'PostconfirmationAdditionalPaidInCapital',
 'PreferredStockIncludingAdditionalPaidInCapital',
 'AdditionalPaidInCapitalCommonStockAndOther',
 'AdditionalPaidInCapitalWarrants',
 'AdditionalPaidInCapitalStockWarrants',
 'AdditionalPaidInCapitalRelatedParty',
 'AdditionalPaidInCapitalCommonStockNetOfAdjustments',
 'AdditionalPaidInCapitalAndAccumulatedDeficitCommonStock',
 'AccumulatedDeficitReclassifiedToAdditionalPaidInCapitalInQuasiReorganization',
 'CommonStockIncludingAdditionalPaidInCapital',
 'PaidInCapitalGeneralUnits',
 'PaidInCapitalLimitedUnits',
 'AdditionalPaidInCapitalCommonStockAndDeferredCompensationEquity',
 'TotalPaidInCapitalAndRetai